In [26]:
import os
import yaml
import time
from tqdm import tqdm
from src.utils import batch_generate, tokens_generate, run_inference
from src.mem import check_memory
from src.instruct import apply_instruct_format
import importlib
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from dotenv import load_dotenv
import plotly.express as px
import torch
import torch.nn.functional as F
import huggingface
from datetime import datetime
load_dotenv('secrets.env')

with open("config/config.yaml", "r") as f: 
    config = yaml.safe_load(f)

model_str = 'deepseek_r1_qwendistill_7'
eval_df = 'big_bench'

ds = config['datasets'][eval_df]
model_path = config['models'][model_str]['path']

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

In [2]:
# !pip freeze > requirements.txt

In [27]:
dataset = load_dataset(ds['source'], ds['subset']).shuffle(config['seed'])
tokenizer = AutoTokenizer.from_pretrained(model_path) 
model = AutoModelForCausalLM.from_pretrained(model_path).to(device)

KeyboardInterrupt: 

# Experiment 1: CoT Determinism (unsure if want to do) 

# Experiment 2: "Check in" (CoT logit analysis)
For each token *t* in our chain of thought, we'll append the </think> tag which triggers a response. From there, we can calculate the probability of a model generating the correct answer at token *t* – this illustrates:
<ul>(1) "when" in the reasoning chain the model starts to arrive at an answer;</ul>
<ul>(2) whether answers are likely to be correct before the model wants to output the </think> tag;</ul>
<ul>(3) whether special tokens like "wait" cause answer probability to improve afterwards.</ul>


In [25]:
df = pd.DataFrame(dataset['train'][0:5])
df['input'] = apply_instruct_format(df['input'], model = model_str, is_answer_format = True)

print(df['input'][0])

<｜begin▁of▁sentence｜><｜User｜>
“Answer the below question. Answer with only the final response without any additional words or phrases. Please reason step by step, and put your final answer within \boxed{}.

 How would a typical person answer each of the following questions about causation?
Janet is an employee in a factory. Since she works in the maintenance department, she knows how to grease and oil all of the machines in the factory. It is her responsibility to put oil into the machines. Kate is also an employee at the factory. While she works in the human resources department, she knows how to grease and oil all of the machines in the factory. If Janet does not put oil in the machines, it is not Kate's responsibility to do so. One day, Janet forgets to put oil in an important machine. Janet noticed that she did not put oil in the machine. Kate did not notice that Janet did not put oil in the machine, and Kate also did not put oil in the machine. The machine broke down a few days la

In [22]:
df = pd.DataFrame(dataset['train'][0:5])
df['input'] = apply_instruct_format(df['input'], model = model_str, is_answer_format = True)

batches = batch_generate(df, ds['input_column'], ds['target_column'])
tokens = tokens_generate(batches, tokenizer, device = device
res = run_inference(model, tokens, tokenizer, time_tracking = True)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation., ?it/s]
inference: 100%|█████████████████████████████████| 1/1 [02:23<00:00, 143.63s/it]


In [23]:
print(res[0]['response'][0])

<｜User｜>
“Answer the below question. Answer with only the final response without any additional words or phrases. Please reason step by step, and put your final answer within \boxed{}.

 How would a typical person answer each of the following questions about causation?
Janet is an employee in a factory. Since she works in the maintenance department, she knows how to grease and oil all of the machines in the factory. It is her responsibility to put oil into the machines. Kate is also an employee at the factory. While she works in the human resources department, she knows how to grease and oil all of the machines in the factory. If Janet does not put oil in the machines, it is not Kate's responsibility to do so. One day, Janet forgets to put oil in an important machine. Janet noticed that she did not put oil in the machine. Kate did not notice that Janet did not put oil in the machine, and Kate also did not put oil in the machine. The machine broke down a few days later. Did Kate not put

In [ ]:
import re
for result in res: 
    for single_response in result['response']:
        cleaned = re.sub(r'^.*?<think>', '<think>', single_response, flags=re.DOTALL)
        print(cleaned)